# TERM SEEKER
Type an English term and get term candidates in the requested UN languages from UN official documents as only source.

## Installation

In [ ]:
import shutil
import os
repo_url = "https://github.com/NelsonJQ/termseeker"
folder_name = "termseeker"

# Your current working directory
#print(os.getcwd()) # if you use your local machine

# if you use Google colab:
%cd /content        

# Remove folder if exists
if os.path.exists(folder_name):
    shutil.rmtree(folder_name)

# Clone the repository
!git clone {repo_url}

%cd termseeker

#!pip3 uninstall termseeker -y

#########################################
# Install the package
#########################################

# If you use your machine and already have all the dependencies installed
# you can install the package without dependencies
#!python3 -m pip install . --no-deps

# If it is the first time you are installing the package
!python3 -m pip install .

## Playground

In [ ]:
# Import the main function
from termseeker import getCandidates, consolidate_results

#########################################
# Define the source term (only one)
input_search_text = "chemical plant"
#########################################

#########################################
# Define the target languages (one or many), only UN languages are supported
input_lang = ["Spanish", "French"]
#input_lang = ["Spanish"]
#########################################

#########################################
# Define the UN document sources to be used (none, one or many)
input_filterSymbols = ["UNEP/EA", "S", "FCCC"]
#########################################

#########################################
# Define the maximum number of source UN documents to be used
# 50 is the limit. 2 or 3 are suggested
sourcesQuantity = 2
#sourcesQuantity = 1
#########################################

#########################################
# Define the number of paragraphs per document to search for the source term
# 1 or 2 are suggested
#paragraphsPerDoc = 2
paragraphsPerDoc = 2
#########################################


# Define if the UN draft documents should be ignored
eraseDrafts = True


# Let us start! The main function will return the result
result= None
result = getCandidates(input_search_text, input_lang, input_filterSymbols, sourcesQuantity, paragraphsPerDoc, eraseDrafts, localLM=True)
print(result)


## Table visualization of results per term

In [ ]:
import polars as pl

# Get the consolidated results as a python list and export its DataFrame to an Excel file
consolidated_results = consolidate_results(result.copy(), exportExcel=False)
print(consolidated_results)
consolidated_df = pl.DataFrame(consolidated_results, strict=False)

# Get a Polars DataFrame from the consolidated results
df = pl.DataFrame(result.copy(), strict=False)
df

In [ ]:
consolidated_df

# Test on uploaded DataFrame

In [ ]:
result_df.write_excel("term2-200Library.xlsx")

In [ ]:
from termseeker.getcandidates import getCandidates
from termseeker.utils import consolidate_results
import polars as pl
import nltk
nltk.download('punkt')

def process_UNTermDF(df):
    """
    Process a UNTerm dataframe by filling in missing language translations.
    
    Args:
        df (pl.DataFrame): DataFrame containing terminology data
        
    Returns:
        pl.DataFrame: DataFrame with filled language data
    """
    
    # Check if the English column exists, else use "term" column
    english_col = "English" if "English" in df.columns else "term"
    if english_col not in df.columns:
        raise ValueError(f"Could not find English term column in the Excel file. Expected 'English' or 'term'.")
    
    # Language columns to check
    languages = ["French", "Spanish", "Russian", "Chinese", "Arabic"]
    
    # Track all keys that appear in any dictionary
    all_keys = set(df.columns)
    
    # Process each row
    processed_data = []
    
    # Check English term for each row, using "term" column as fallback
    for i, row in enumerate(df.iter_rows(named=True)):
        row_dict = dict(row)
        
        # First try to get value from "English" column, fall back to "term" if empty
        english_term = None
        if "English" in row_dict and row_dict["English"] and row_dict["English"] != "None":
            english_term = row_dict["English"]
        elif "term" in row_dict and row_dict["term"] and row_dict["term"] != "None":
            english_term = row_dict["term"]
            # If we're using the term column, ensure it's also set in the English field
            row_dict["English"] = english_term
        
        # Skip if no English term found in either column
        if not english_term:
            processed_data.append(row_dict)
            continue
        
        # Identify missing languages
        missing_langs = []
        for lang in languages:
            # Check if language column exists and if value is missing
            if lang in row_dict and (row_dict[lang] is None or row_dict[lang] == "" or row_dict[lang] == "None"):
                missing_langs.append(lang)
        
        # If there are missing languages, use getCandidates to fill them
        if missing_langs:
            print(f"Processing term ({i+1}/{len(df)}): '{english_term}' - Missing languages: {missing_langs}")
            
            try:
                # Get candidates using getCandidates
                results = getCandidates(
                    input_search_text=english_term,
                    input_lang=missing_langs,
                    #input_filterSymbols=["UNEP/EA", "S", "FCCC"],
                    input_filterSymbols=["UNEP/EA"],
                    sourcesQuantity=2,
                    paragraphsPerDoc=2,
                    eraseDrafts=True,
                    localLM=None
                )
                
                # Consolidate results if we got any
                if results:
                    consolidated = consolidate_results(results, exportExcel=False)
                    
                    # If consolidated results available, merge with row data
                    if consolidated and len(consolidated) > 0:
                        # Extract first item (dictionary) from consolidated results
                        result_dict = consolidated[0]
                        
                        # Add all fields from consolidated results to row_dict
                        for key, value in result_dict.items():
                            if value is not None:
                                # Add this key to the set of all keys
                                all_keys.add(key)
                                # Add as new column regardless of whether it exists in original data
                                row_dict[key] = value
            except Exception as e:
                print(f"Error processing term '{english_term}': {str(e)}")
        
        processed_data.append(row_dict)

    return processed_data

# Get polars DataFrame from Excel file
path_file = "C:\\Users\\Nelso\\Documents\\TermhUNter\\terms2-1935.xlsx"
df = pl.read_excel(path_file)

# slice df to rows 200-320
#df = df.head(120)
df = df.slice(1400, 200)
#df = df.tail(2)

results = process_UNTermDF(df)


# Find all possible keys across all dictionaries
all_possible_keys = set()
for row_dict in results:
    all_possible_keys.update(row_dict.keys())

# Normalize all dictionaries to have the same keys
normalized_data = []
for row_dict in results:
    normalized_dict = {key: row_dict.get(key, None) for key in all_possible_keys}
    normalized_data.append(normalized_dict)

# Create a new DataFrame from the normalized data
result_df = pl.DataFrame(normalized_data)

result_df

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nelso\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processing term (1/200): 'livestock farming' - Missing languages: ['French', 'Spanish', 'Russian', 'Chinese', 'Arabic']
Request was successful. Content:
Found 0 potential documents

--- Language paragraph counts ---
French: 0 paragraphs
Spanish: 0 paragraphs
Russian: 0 paragraphs
Chinese: 0 paragraphs
Arabic: 0 paragraphs
Processing term (2/200): 'living condition' - Missing languages: ['French', 'Spanish', 'Russian', 'Chinese', 'Arabic']
Request was successful. Content:
Found 5 potential documents
Modified 0 out of 5 symbols. Removed whitespaces from 0 and hyphens from 0. Filtered out 2 items with 'draft' in docType, and 1 with no translations available.
After cleaning, 2 documents remain for processing
Processing document 1/2: UNEP/EA.2/RES.16
Processing files for https://daccess-ods.un.org/access.nsf/Get?OpenAgent&DS=UNEP/EA.2/RES.16&Lang=E...
		convert.py -> got response
		convert.py -> using pymupdf4llm
Processing C:\Users\Nelso\AppData\Local\Temp\tmptktgkynt.pdf...
[             

SpanishSynonyms,UNTerm_Source,ArabicParagraphs,Russian,English,FrenchSynonyms,docTitle,Arabic,docType,RussianSynonyms,EnglishTerm,isUnterm,ArabicSynonyms,ChineseSynonyms,docSymbol,EnglishParagraphs,ChineseParagraphs,FrenchParagraphs,publicationDate,Chinese,SpanishParagraphs,RussianParagraphs,term,French,Spanish,error
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""{'Synonyms': [], 'Similar': []…","""None""",null,null,"""livestock farming""","""{'Synonyms': [], 'Similar': []…",null,null,null,"""{'Synonyms': [], 'Similar': []…",null,"""NotFound""","""{'Synonyms': [], 'Similar': []…","""{'Synonyms': [], 'Similar': []…",null,null,null,null,null,null,null,null,null,null,null,null
"""{'Synonyms': [], 'Similar': ['…","""None""","""#### الرفاه تعميم مراعاة التنو…",null,"""living condition""","""{'Synonyms': [], 'Similar': ['…","""2/16. Mainstreaming of biodive…",null,"""Resolutions and Decisions Repo…","""{'Synonyms': [], 'Similar': ['…","""living condition""","""NotFound""","""{'Synonyms': [], 'Similar': ['…","""{'Synonyms': [], 'Similar': ['…","""UNEP/EA.2/RES.16 UNEP/EA.2/19""","""3. _Stresses that the integrat…","""3. 强调指出《2030 年可持续发展议程》中确立的政策一体…","""3. _Affirme que l’intégration …","""2016-08-04 2016-07-01""",null,"""3. _Subraya que la integración…","""3. _подчеркивает, что интеграц…",null,null,null,null
"""None""","""None""",null,null,"""living marine resource""","""None""",null,null,null,"""None""",null,"""NotFound""","""None""","""None""",null,null,null,null,null,null,null,null,"""living marine resource""",null,null,"""Search error: cannot access lo…"
"""{'Synonyms': [], 'Similar': ['…","""{'source': 'UNHQ', 'tags': ['O…",null,"""прикомандирование""","""loan""","""{'Synonyms': [], 'Similar': ['…",null,"""إعارة""",null,"""{'Synonyms': [], 'Similar': ['…",null,"""UNTerm""","""{'Synonyms': [], 'Similar': ['…","""{'Synonyms': [], 'Similar': ['…",null,null,null,null,null,"""借调""",null,null,null,"""prêt""","""préstamo""",null
"""{'Synonyms': [], 'Similar': ['…","""None""",null,null,"""locust""","""{'Synonyms': [], 'Similar': ['…",null,null,null,"""{'Synonyms': [], 'Similar': ['…",null,"""NotFound""","""{'Synonyms': [], 'Similar': ['…","""{'Synonyms': [], 'Similar': ['…",null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""{'Synonyms': [], 'Similar': ['…","""None""",null,null,"""phosphate""","""{'Synonyms': [], 'Similar': ['…",null,null,null,"""{'Synonyms': [], 'Similar': ['…",null,"""NotFound""","""{'Synonyms': [], 'Similar': ['…","""{'Synonyms': [], 'Similar': ['…",null,null,null,null,null,null,null,null,null,null,null,null
"""{'Synonyms': [], 'Similar': ['…","""None""",null,null,"""photochemical oxidant""","""{'Synonyms': [], 'Similar': ['…",null,null,null,"""{'Synonyms': [], 'Similar': ['…",null,"""NotFound""","""{'Synonyms': [], 'Similar': ['…","""{'Synonyms': [], 'Similar': ['…",null,null,null,null,null,null,null,null,null,null,null,null
"""{'Synonyms': [], 'Similar': ['…","""None""",null,null,"""photograph""","""{'Synonyms': [], 'Similar': ['…",null,null,null,"""{'Synonyms': [], 'Similar': ['…",null,"""NotFound""","""{'Synonyms': [], 'Similar': ['…","""{'Synonyms': [], 'Similar': ['…",null,null,null,null,null,null,null,null,null,null,null,null


In [6]:
#export df to excel
result_df.write_excel("test1400-1600UNEAonly.xlsx")

In [7]:
df = pl.read_excel(path_file)
df = df.slice(1600, 300)
#df = df.tail(2)

results = process_UNTermDF(df)


# Find all possible keys across all dictionaries
all_possible_keys = set()
for row_dict in results:
    all_possible_keys.update(row_dict.keys())

# Normalize all dictionaries to have the same keys
normalized_data = []
for row_dict in results:
    normalized_dict = {key: row_dict.get(key, None) for key in all_possible_keys}
    normalized_data.append(normalized_dict)

# Create a new DataFrame from the normalized data
result_df = pl.DataFrame(normalized_data)

result_df.write_excel("test1600-1900UNEAonly.xlsx")

Processing term (1/280): 'pilgrimage' - Missing languages: ['French', 'Spanish', 'Russian', 'Chinese', 'Arabic']


An error occurred: HTTPSConnectionPool(host='digitallibrary.un.org', port=443): Read timed out. (read timeout=None)
General term search without filters...
An error occurred: HTTPSConnectionPool(host='digitallibrary.un.org', port=443): Read timed out. (read timeout=None)

--- Language paragraph counts ---
French: 0 paragraphs
Spanish: 0 paragraphs
Russian: 0 paragraphs
Chinese: 0 paragraphs
Arabic: 0 paragraphs
Processing term (3/280): 'pipeline' - Missing languages: ['French', 'Spanish', 'Russian', 'Chinese', 'Arabic']
An error occurred: HTTPSConnectionPool(host='digitallibrary.un.org', port=443): Read timed out. (read timeout=None)
General term search without filters...
An error occurred: HTTPSConnectionPool(host='digitallibrary.un.org', port=443): Read timed out. (read timeout=None)

--- Language paragraph counts ---
French: 0 paragraphs
Spanish: 0 paragraphs
Russian: 0 paragraphs
Chinese: 0 paragraphs
Arabic: 0 paragraphs
Processing term (4/280): 'planet' - Missing languages: ['Fren

## Debugging

In [ ]:
from termun.utils import *
from termun.convert import *

ESPpdf_path = "n1903161.pdf"
ENGpdf_path = "n1903158.pdf"

# Extract text from PDF
mdES = convert_pdf_to_markdown(ESPpdf_path)
mdEN = convert_pdf_to_markdown(ENGpdf_path)
print(mdEN)

In [ ]:
engParatest = find_paragraphs_with_merge(mdEN, "enhancing mutual trust and removing", max_paragraphs=2)
print(engParatest)
espParagraphs = find_similar_paragraph_in_target(engParatest[0],
                                                 mdES,
                                                 model_name='distiluse-base-multilingual-cased-v2',
                                                 top_k=2)


print("\n\n")
print(espParagraphs)

In [ ]:
from termun.utils import *
from termun.convert import *
testkw = "atmosphere begs"
engParas = find_paragraphs_with_merge2(mdEN, testkw, 2)
espParas = find_paragraphs_with_merge2(mdES, "hecho que ha sido condenado, incluso", 2)
print(espParas)
print(engParas)

In [ ]:
import polars as pl

file1 = "../test200-320UNEAonly.xlsx"
file2 = "../test320-620UNEAonly.xlsx"
file3 = "../test620-800UNEAonly.xlsx"

# Read Excel files as polars DataFrames
df1 = pl.read_excel(file1)
df2 = pl.read_excel(file2)
df3 = pl.read_excel(file3)

# Option 1: Use union of columns (keep all columns)
# Get all unique columns from all dataframes
all_columns = set()
for df in [df1, df2, df3]:
    all_columns.update(df.columns)

# Make sure each dataframe has all columns
dfs = []
for df in [df1, df2, df3]:
    # Add missing columns with null values
    for col in all_columns:
        if col not in df.columns:
            df = df.with_columns(pl.lit(None).alias(col))
    # Ensure columns are in the same order
    dfs.append(df.select(sorted(all_columns)))

# Concatenate the normalized DataFrames
result_df = pl.concat(dfs)

# Alternative Option 2: Use intersection of columns (only common columns)
# Get columns that exist in all dataframes
common_columns = set(df1.columns)
for df in [df2, df3]:
    common_columns &= set(df.columns)

# Select only common columns in the same order
if common_columns:
    common_columns_list = sorted(common_columns)
    result_df_common = pl.concat([
        df.select(common_columns_list) for df in [df1, df2, df3]
    ])

result_df

In [ ]:
# Export df to excel
result_df.write_excel("compiled2-800UNEAonly.xlsx")

In [ ]:
# print df structure, column names
print(result_df.schema)

## Multilingual LLM prompt for batch extraction

In [ ]:
from openai import OpenAI
import polars as pl
import ast
import json
import os
from groq import Groq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
api_key = os.environ.get("GROQ_API_KEY")

def askLLM_batchextraction(sourceTerm, sourceLanguage="English", 
                          contexts=None, url='http://localhost:1234/v1'):
    """
    Generate translations of a source term into multiple languages using local LLM API with structured JSON output.
    
    Args:
        sourceTerm (str): The term to translate
        sourceLanguage (str): The language of the source term (default: "English")
        contexts (dict): Dictionary containing context for each target language with format:
                        {
                            "ES": {"context": str, "similar": str, "synonyms": list},
                            "FR": {"context": str, "similar": str, "synonyms": list},
                            ...
                        }
        url (str): The base URL of the local language model API (default: 'http://localhost:1234/v1')
    
    Returns:
        dict: JSON response with translations in multiple languages
    """
    
    # Initialize OpenAI client with local endpoint
    #client = OpenAI(base_url=url, api_key="lm-studio")
    client = Groq()

    # Language codes and their full names
    language_map = {
        "ES": "Spanish",
        "FR": "French", 
        "CH": "Chinese", 
        "RU": "Russian", 
        "AR": "Arabic"
    }
    
    # Create the prompt as a nested dictionary (will be converted to JSON)
    prompt_data = {
        "sourceTerm": sourceTerm,
        "sourceLanguage": sourceLanguage,
        "outputFormat": "JSON",
        "outputStyle": {
            "type": "json_schema",
            "json_schema": {
                "name": "translations",
                "schema": {
                    "type": "object",
                    "properties": {
                        "terms": {
                            "type": "object",
                            "properties": {
                                "English": {
                                    "type": "string",
                                    "description": f"The original {sourceLanguage} term being translated",
                                    "enum": [sourceTerm]  # This restricts it to exactly the sourceTerm value
                                },
                                "Français": {
                                    "type": "array",
                                    "items": {"type": "string"},
                                    "minItems": 1,
                                    "maxItems": 4,
                                    "description": f"List of French translations for the term '{sourceTerm}' based on the context provided"
                                },
                                "Español": {
                                    "type": "array",
                                    "items": {"type": "string"},
                                    "minItems": 1,
                                    "maxItems": 4,
                                    "description": f"List of Spanish translations for the term '{sourceTerm}' based on the context provided"
                                },
                                "简体中文": {
                                    "type": "array",
                                    "items": {"type": "string"},
                                    "minItems": 1,
                                    "maxItems": 4,
                                    "description": f"List of Simplified Chinese translations for the term '{sourceTerm}' based on the context provided"
                                },
                                "Русский": {
                                    "type": "array",
                                    "items": {"type": "string"},
                                    "minItems": 1,
                                    "maxItems": 4,
                                    "description": f"List of Russian translations for the term '{sourceTerm}' based on the context provided"
                                },
                                "العربية": {
                                    "type": "array",
                                    "items": {"type": "string"},
                                    "minItems": 1,
                                    "maxItems": 4,
                                    "description": f"List of Arabic translations for the term '{sourceTerm}' based on the context provided"
                                }
                            },
                            "required": ["English", "Español", "简体中文", "Français", "Русский", "العربية"]
                        }
                    },
                    "required": ["terms"]
                }
            }
        }
    }

    # Prepare context information for each language
    contexts = contexts or {}
    
    for lang_code, lang_name in language_map.items():
        context_info = contexts.get(lang_code, {})
        context_text = context_info.get("context", "")
        similar_text = context_info.get("similar", "")
        synonyms_list = context_info.get("synonyms", [])
        
        # Add context to the prompt data
        prompt_data[f"{lang_name}Context"] = {
            "Synonyms": synonyms_list,
            "documents": f"{context_text}\n\n{similar_text}".strip()
        }
    
    # Convert to JSON string for the prompt
    prompt_json = json.dumps(prompt_data, ensure_ascii=False, indent=2)
    
    # Define the response format for structured output
    response_format = {
        "type": "json_object",
        "json_object": {
            "name": "translations",
            "schema": prompt_data["outputStyle"]["json_schema"]["schema"]
        }
    }
    
    try:
        
        final_prompt = f"Suggest a translation for <sourceterm>{sourceTerm}</sourceterm> in <targetlanguages>Russian, Spanish, Arabic, French, Simplified Chinese</targetlanguages> based on mentions of contextual documents and synonyms here below. Do not provide similar terms but the proper translation of source English string." + prompt_json
        # Create a chat completion
        completion = client.chat.completions.create(
            #model="model-identifier",  # not essential for LM Studio
            model="llama-3.3-70b-versatile",
            messages=[
                {"role": "system", "content": "You are a helpful multilingual assistant that understands English, French, Simplified Chinese, Arabic, Russian and Spanish. You suggest accurate translations of a single input term based on provided context."},
                {"role": "user", "content": final_prompt}
            ],
            temperature=0.05,
            response_format=response_format,
            max_completion_tokens=1230,
            stream=False,
            stop=None,
            top_p=1
        )
        

        # Parse the response as JSON
        #response_content = completion.choices[0].message.content
        response_content = completion.choices[0].message.content
        
        return json.loads(response_content)
    
    except Exception as e:
        print(f"Error in LLM batch extraction: {str(e)}")
        return {"error": str(e)}


def process_dataframe_for_term_extraction(df, url='http://localhost:1234/v1'):
    """
    Process a polars dataframe and extract translations for each term using askLLM_batchextraction.
    Includes all rows in the output, with None for translation_result when EnglishTerm is missing.
    
    Args:
        df: Polars DataFrame with language context information and English terms
        url: The base URL of the local language model API (default: 'http://localhost:1234/v1')
        
    Returns:
        List of row dictionaries with translation results added
    """
    results = []
    
    # Convert to pandas for row iteration if it's a LazyFrame
    if isinstance(df, pl.LazyFrame):
        df = df.collect()
    
    # Convert to dictionaries for row processing
    rows = df.to_dicts()
    
    for idx, row in enumerate(rows):
        # Get a copy of the row data
        row_data = row.copy()
        
        # Check if English term exists
        english_term = row.get('EnglishTerm', '')
        if english_term is None or english_term == '':
            print(f"Skipping translation for row {idx}: Missing English term")
            # Add the row with None translation_result
            row_data['translation_result'] = None
            results.append(row_data)
            continue
            
        # Get the source term
        source_term = english_term
        print(f"Processing term: {source_term}")
        
        # Prepare contexts for each language
        contexts = {}
        
        # Language codes
        language_codes = ['AR', 'CH', 'FR', 'RU', 'ES']
        
        for lang_code in language_codes:
            # Map language code to full name as in the dataframe
            lang_map = {
                'AR': 'Arabic',
                'CH': 'Chinese',
                'FR': 'French',
                'RU': 'Russian',
                'ES': 'Spanish'
            }
            lang_name = lang_map[lang_code]
            
            # Get context from paragraphs
            context_text = row.get(f"{lang_name}Paragraphs", "")
            if context_text is None:
                context_text = ""
            
            # Parse synonyms information
            synonyms_list = []
            similar_text = ""
            
            synonyms_data = row.get(f"{lang_name}Synonyms", "")
            if synonyms_data is not None and synonyms_data != "":
                try:
                    # Try to parse the synonyms data safely
                    if isinstance(synonyms_data, str):
                        try:
                            synonyms_dict = ast.literal_eval(synonyms_data)
                        except:
                            try:
                                synonyms_dict = json.loads(synonyms_data)
                            except:
                                print(f"Warning: Could not parse {lang_name}Synonyms data")
                                synonyms_dict = {}
                    else:
                        # If it's already a dict, use it directly
                        synonyms_dict = synonyms_data if isinstance(synonyms_data, dict) else {}
                    
                    synonyms_list = synonyms_dict.get('Synonyms', [])
                    similar_items = synonyms_dict.get('Similar', [])
                    similar_text = ", ".join(similar_items) if isinstance(similar_items, list) else str(similar_items)
                    
                except Exception as e:
                    print(f"Error processing {lang_name}Synonyms: {e}")
            
            # Add to contexts
            contexts[lang_code] = {
                "context": context_text,
                "similar": similar_text,
                "synonyms": synonyms_list
            }
        
        # Call the extraction function
        try:
            translation_result = askLLM_batchextraction(
                sourceTerm=source_term,
                sourceLanguage="English",
                contexts=contexts,
                url=url
            )
            
            # Add the result to the row data
            row_data['translation_result'] = json.dumps(translation_result)  # Serialize to JSON string
            results.append(row_data)
            
        except Exception as e:
            print(f"Error in term extraction for {source_term}: {e}")
            # Still add the row but with error information
            row_data['translation_result'] = json.dumps({"error": str(e)})  # Serialize to JSON string
            results.append(row_data)
    
    return results


In [ ]:
result_df = pl.read_excel("../compiled200-800UNEAonly.xlsx")

# print row index 559
test_df = result_df.slice(520, 15)
print(test_df)

In [ ]:
# slice df to tail 2

processed_results = process_dataframe_for_term_extraction(test_df)


#processed_df = pl.DataFrame(processed_results)

In [ ]:
processed_df = pl.DataFrame(processed_results)
processed_df.write_excel("test7LMoutput_Groq15.xlsx")

#{"terms": {"SpanishTerm": ["reconstrucci\u00f3n econ\u00f3mica", "recuperaci\u00f3n econ\u00f3mica", "programa de relance \u00e9conomique", "programa de recuperaci\u00f3n econ\u00f3mica"], "ChineseTerm": ["\u7ecf\u6d4e\u590d\u82cf\u65b9\u6848", "\u7d27\u6025\u7ecf\u6d4e\u590d\u82cf\u65b9\u6848", "\u590d\u82cf\u65b9\u6848", "\u7ecf\u6d4e\u6301\u7eed\u590d\u82cf"], "RussianTerm": ["\u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0430 \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u0447\u0435\u0441\u043a\u043e\u0433\u043e \u0432\u043e\u0441\u0441\u0442\u0430\u043d\u043e\u0432\u043b\u0435\u043d\u0438\u044f", "\u0446\u044e\u0440\u0435\u0437\u0432\u044b\u0447\u0430\u0439\u043d\u0430\u044f \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0430 \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u0447\u0435\u0441\u043a\u043e\u0433\u043e \u0432\u043e\u0441\u0441\u0442\u0430\u043d\u043e\u0432\u043b\u0435\u043d\u0438\u044f", "\u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0430 \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u0447\u0435\u0441\u043a\u043e\u0433\u043e \u0432\u043e\u0441\u0441\u0442\u0430\u043d\u043e\u0432\u043b\u0435\u043d\u0438\u044f", "\u0446\u044e\u0440\u0435\u0437\u0432\u044b\u0447\u0430\u0439\u043d\u0430\u044f \u043f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0430 \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u0447\u0435\u0441\u043a\u043e\u0433\u043e \u0432\u043e\u0441\u0441\u0442\u0430\u043d\u043e\u0432\u043b\u0435\u043d\u0438\u044f"], "ArabicTerm": ["\u0628\u0631\u0646\u0627\u0645\u062c \u0627\u0644\u0625\u0646\u0639\u0627\u0634 \u0627\u0644\u0627\u0642\u062a\u0635\u0627\u062f\u064a", "\u0628\u0631\u0646\u0627\u0645\u062c \u0627\u0644\u0625\u0646\u0639\u0627\u0634 \u0627\u0644\u0627\u0642\u062a\u0635\u0627\u062f\u064a \u0644\u0644\u0637\u0648\u0627\u0631\u0626", "\u0627\u0646\u062a\u0639\u0627\u0634 \u0627\u0642\u062a\u0635\u0627\u062f\u064a \u0645\u062a\u0648\u0627\u0635\u0644", "\u0627\u0646\u062a\u0639\u0627\u0634 \u0627\u0642\u062a\u0635\u0627\u062f\u064a \u0645\u064f\u0633\u062a\u062f\u064a\u0645"], "FrenchTerm": ["programme de rel\u00e8vement \u00e9conomique", "programme de relance \u00e9conomique d'urgence", "programme de r\u00e9cup\u00e9ration \u00e9conomique", "programme de rel\u00e8vement \u00e9conomique"]}}

In [ ]:
def parse_translation_results(df):
    """Parse JSON results and create a more readable DataFrame"""
    results = []
    
    for row in df.iter_rows(named=True):
        try:
            # Get original data
            english_term = row.get('EnglishTerm', 'Unknown')
            
            # Parse the JSON
            if row['translation_result'] is None:
                continue
                
            translation = json.loads(row['translation_result'])
            terms = translation.get('terms', {})
            
            # Create a row for each language
            for lang, term_list in terms.items():
                # Convert list to string for better compatibility
                if isinstance(term_list, list):
                    translations_str = ", ".join(term_list)
                else:
                    translations_str = str(term_list)
                    
                results.append({
                    'EnglishTerm': english_term,
                    'Language': lang,
                    'Translations': translations_str
                })
                
        except Exception as e:
            print(f"Error parsing row: {e}")
    
    # Create a new DataFrame with parsed results
    return pl.DataFrame(results)

# Create a more readable DataFrame
readable_results = parse_translation_results(processed_df)
readable_results.write_excel("test15Groq_parsed.xlsx")
readable_results

# For a specific row (e.g., the first row)
#row_json = processed_df[2, 'translation_result']
#parsed_json = json.loads(row_json)

# Print in readable format
#import pprint
#pprint.pprint(parsed_json, width=100, sort_dicts=False)

### Expand df with LLM responses

In [ ]:
def expand_translation_results(df):
    """Parse JSON results and expand them into new columns"""
    # First convert JSON strings to dictionaries
    parsed_rows = []
    
    for row in df.iter_rows(named=True):
        try:
            row_dict = dict(row)
            translation = json.loads(row['translation_result'])
            
            # Extract each language's terms
            for lang, terms in translation.get('terms', {}).items():
                # Join terms with comma for display
                row_dict[lang] = ", ".join(terms)
                
                # Also keep individual terms if needed
                for i, term in enumerate(terms, 1):
                    row_dict[f"{lang}_{i}"] = term
            
            parsed_rows.append(row_dict)
        except Exception as e:
            print(f"Error parsing row: {e}")
            parsed_rows.append(row)
    
    # Create a new DataFrame with expanded columns
    return pl.DataFrame(parsed_rows)

# Create expanded DataFrame
expanded_df = expand_translation_results(processed_df)

# Export to excel
expanded_df.write_excel("test7LMoutput_expanded.xlsx")
expanded_df